# SQL DataBase Parsing

In [2]:
## create sample SqLite Database
import sqlite3
import os

os.makedirs("data/databases", exist_ok = True)

In [3]:
## create sample

conn = sqlite3.connect('data/databases/company.db')
cursor = conn.cursor()

In [4]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        role TEXT,
        department TEXT,
        salary REAL
    )
''')

In [5]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS projects (
        id INTEGER PRIMARY KEY,
        name TEXT,
        status TEXT,
        budget REAL,
        lead_id INTEGER
    )
''')

In [7]:
employees = [
    (1, 'John Doe', 'Senior Developer', 'Engineering', 95000),
    (2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000),
    (3, 'Mike Johnson', 'Product Manager', 'Product', 110000),
    (4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000)
]

projects = [
    (1, 'RAG Implementation', 'Active', 150000, 1),
    (2, 'Data Pipeline', 'Completed', 80000, 2),
    (3, 'Customer Portal', 'Planning', 200000, 3),
    (4, 'ML Platform', 'Active', 250000, 2)
]

In [8]:
cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?, ?)', employees)
cursor.executemany('INSERT INTO projects VALUES (?, ?, ?, ?, ?)', projects)

In [9]:
cursor.execute("Select * from employees")

In [10]:
conn.commit()
conn.close()

## DataBase Document Extraction

In [13]:
from langchain_community.utilities import SQLDatabase

from langchain_community.document_loaders import SQLDatabaseLoader


In [16]:
db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

print(db.get_usable_table_names())
print(db.get_table_info())

['employees', 'projects']

CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	John Doe	Senior Developer	Engineering	95000.0
2	Jane Smith	Data Scientist	Analytics	105000.0
3	Mike Johnson	Product Manager	Product	110000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	RAG Implementation	Active	150000.0	1
2	Data Pipeline	Completed	80000.0	2
3	Customer Portal	Planning	200000.0	3
*/


## Custom SQL Processing

In [ ]:
from typing import List
from langchain_core.documents import Document


def sql_to_doc(db_path : str) -> List[Document]:

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    documents = []

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")    
    tables = cursor.fetchall()   


    for table in tables:
        table_name = table[0]

        # Get column names
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        column_names = [col[1] for col in columns]
        # Fetch all rows
        cursor.execute(f"SELECT * FROM {table_name};")
        rows = cursor.fetchall()

        table_content = f"Table: {table_name}\nColumns: {', '.join(column_names)}\n\n"

        table_content += "Sample Records:\n"
        for row in rows[:5]:  # Limiting to first 5 rows for brev
            record = dict(zip(column_names, row))
            table_content += f"{record}\n"

        doc = Document(page_content=table_content, metadata={"source": table_name,
                                                             "table_name": table_name,
                                                             "num_records": len(rows),
                                                             "data_type": "sql"
                                                             })
        documents.append(doc)


        cursor.execute("""
                       SELECT e.name, e.role, p.name as project_name, p.status
                       FROM employees e 
                       JOIN projects p ON e.id = p.lead_id
                       """)
        relationships = cursor.fetchall()
        rel_content = "Employee-Project Relationships \n\n"

        for rel in relationships:
            rel_content += f"{rel[0]} ({rel[1]}) leads {rel[2]} - Status {rel[3]} \n"


        rel_doc = Document(
            page_content=rel_content,
            metadata = {
                'source' : db_path,
                'db_path' : 'sql_rel'
            }
        )

        documents.append(rel_doc)

    return documents


In [18]:
sql_to_doc("data/databases/company.db")

[Document(metadata={'source': 'employees', 'table_name': 'employees', 'num_records': 4, 'data_type': 'sql'}, page_content="Table: employees\nColumns: id, name, role, department, salary\n\nSample Records:\n{'id': 1, 'name': 'John Doe', 'role': 'Senior Developer', 'department': 'Engineering', 'salary': 95000.0}\n{'id': 2, 'name': 'Jane Smith', 'role': 'Data Scientist', 'department': 'Analytics', 'salary': 105000.0}\n{'id': 3, 'name': 'Mike Johnson', 'role': 'Product Manager', 'department': 'Product', 'salary': 110000.0}\n{'id': 4, 'name': 'Sarah Williams', 'role': 'DevOps Engineer', 'department': 'Engineering', 'salary': 98000.0}\n"),
 Document(metadata={'source': 'projects', 'table_name': 'projects', 'num_records': 4, 'data_type': 'sql'}, page_content="Table: projects\nColumns: id, name, status, budget, lead_id\n\nSample Records:\n{'id': 1, 'name': 'RAG Implementation', 'status': 'Active', 'budget': 150000.0, 'lead_id': 1}\n{'id': 2, 'name': 'Data Pipeline', 'status': 'Completed', 'budg